In [ ]:
import pandas as pd
import os
import json
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
from scripts.models import FacePrediction
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob
import tensorflow

In [ ]:
bs = 16
eps = 20
model_type = 'vgg16' # 'vgg16' 'vgg16_fc6'
model_dir = './ML Final Project/BMI/Models/'
img_dir = './ML Final Project/BMI/Data/Images/'
train_dir = './ML Final Project/BMI/Data/Train Images/'
test_dir = './ML Final Project/BMI/Data/Test Images/'

allimages = os.listdir(img_dir)
train = pd.read_csv('./ML Final Project/BMI/Data/train.csv')
valid = pd.read_csv('./ML Final Project/BMI/Data/test.csv')

train = train.loc[(train['is_training'] == 1) & (train['name'].isin(allimages))]
print('Train Shape:', train.shape)
valid = valid.loc[(valid['is_training'] == 0) & (valid['name'].isin(allimages))]
print('Test Shape:',valid.shape)

## Model Training

In [ ]:
es = EarlyStopping(monitor = 'val_bmi_loss', mode = 'min', patience=3)
ckp = ModelCheckpoint(model_dir, monitor = 'val_bmi_loss', mode = 'min', save_best_only=True, save_weights_only=False, verbose=1)
callbacks = [es, ckp]

model = FacePrediction(img_dir = img_dir, model_type = model_type)
model.define_model()
model.model.summary()

model_history = model.train(train, valid, bs = bs, epochs = eps, callbacks = callbacks)
model.save_model(model_id)

In [ ]:
train_loss = model_history.history['bmi_loss']
val_loss = model_history.history['val_bmi_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'b', label='Train Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Train and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
metrics = model.evaluate(valid)

metrics['model'] = model_id
metrics['epoch'] = eps  # Add epoch value
metrics['batch_size'] = bs  # Add batch size value
metrics

In [ ]:
preds = model.predict_faces('./ML Final Project/BMI/Data/Images/img_390.bmp', show_img = True)

In [58]:
preds = model.rt_predict_faces(color = "white")

1/1 [==============================] - 0s 199ms/step
